# P2 | Credit Card Revival

## Introduction

This midbootcamp project will focus in a classification problem: as a risk analyst of a bank I will analyse and understand the demographics and other characteristics of the bank customer's that accept or reject a credit card offer, using linear regression.

## Import Libraries